In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from diff_skewed_gaussian_rasterization import GaussianRasterizer, GaussianRasterizationSettings
from PIL import Image
import ipywidgets as widgets
from ipywidgets import interactive, fixed, FloatSlider, IntSlider, ColorPicker
from IPython.display import display, clear_output

# Comprobamos si hay GPU disponible
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

Usando dispositivo: cuda:0


In [4]:
# Configuración para la renderización
def create_raster_settings(camera_x, camera_y, camera_z, 
                          look_x, look_y, look_z,
                          up_x, up_y, up_z,
                          fov_degrees, bg_color,
                          scale_modifier, image_size):
    
    # Convertir FOV de grados a radianes
    fov_rad = np.radians(fov_degrees)
    tan_fov = np.tan(fov_rad / 2)
    
    # Configurar la cámara
    eye = np.array([camera_x, camera_y, camera_z])
    target = np.array([look_x, look_y, look_z])
    up = np.array([up_x, up_y, up_z])
    
    # Normalizar el vector up
    up = up / np.linalg.norm(up)
    
    # Crear matrices de vista y proyección
    view_matrix = get_camera_matrix(eye, target, up).to(device)
    proj_matrix = get_projection_matrix(fov_rad, fov_rad).to(device)
    campos = torch.tensor(eye, dtype=torch.float32, device=device)
    
    # Color de fondo
    bg = torch.tensor(bg_color, dtype=torch.float32, device=device)
    
    # Crear los parámetros de configuración del rasterizador
    raster_settings = GaussianRasterizationSettings(
        image_height=image_size,
        image_width=image_size,
        tanfovx=tan_fov,
        tanfovy=tan_fov,
        bg=bg,
        scale_modifier=scale_modifier,
        viewmatrix=view_matrix,
        projmatrix=proj_matrix,
        sh_degree=0,
        campos=campos,
        prefiltered=False,
        debug=True,  # Activar modo debug para ver más información
    )
    
    return raster_settings, eye, target, up

# Crear una matriz de vista simple
def get_camera_matrix(eye, target, up):
    """Crea una matriz de cámara mirando desde 'eye' hacia 'target'"""
    eye_tensor = torch.tensor(eye, dtype=torch.float32)
    target_tensor = torch.tensor(target, dtype=torch.float32)
    up_tensor = torch.tensor(up, dtype=torch.float32)
    
    # Asegurarse de que los vectores no sean paralelos
    if np.allclose(eye, target):
        target = target + np.array([0.0, 0.0, 0.1])
        target_tensor = torch.tensor(target, dtype=torch.float32)
    
    z = eye_tensor - target_tensor
    z = z / (torch.norm(z) + 1e-10)  # Evitar división por cero
    
    x = torch.cross(up_tensor, z)
    x = x / (torch.norm(x) + 1e-10)  # Evitar división por cero
    
    y = torch.cross(z, x)
    y = y / (torch.norm(y) + 1e-10)  # Normalizar por seguridad
    
    view_matrix = torch.eye(4, dtype=torch.float32)
    view_matrix[0, :3] = x
    view_matrix[1, :3] = y
    view_matrix[2, :3] = z
    view_matrix[:3, 3] = -torch.tensor([torch.dot(x, eye_tensor),
                                        torch.dot(y, eye_tensor),
                                        torch.dot(z, eye_tensor)])
    
    return view_matrix

# Crear matriz de proyección
def get_projection_matrix(fovx, fovy, near=0.1, far=100.0):
    """Crea una matriz de proyección perspectiva"""
    proj_matrix = torch.zeros(4, 4, dtype=torch.float32)
    
    # Proyección perspectiva estándar
    proj_matrix[0, 0] = 1.0 / np.tan(fovx / 2)
    proj_matrix[1, 1] = 1.0 / np.tan(fovy / 2)
    proj_matrix[2, 2] = -(far + near) / (far - near)
    proj_matrix[2, 3] = -2 * far * near / (far - near)
    proj_matrix[3, 2] = -1.0
    
    return proj_matrix

# Calcular las posiciones 2D proyectadas con corrección de profundidad
def project_points(points, view_matrix, proj_matrix):
    """Proyecta puntos 3D a coordenadas 2D"""
    # Convertir a coordenadas homogéneas
    homogeneous = torch.ones((points.shape[0], 4), device=points.device)
    homogeneous[:, :3] = points
    
    # Aplicar transformación view-projection
    view_proj = torch.matmul(proj_matrix, view_matrix)
    projected = torch.matmul(homogeneous, view_proj.T)
    
    # Dividir por w para obtener coordenadas NDC
    # Usar un valor pequeño para evitar división por cero
    projected_w = projected[:, 3:4].clone()
    projected_w[projected_w.abs() < 1e-6] = 1e-6  # Evitar división por cero
    ndc = projected[:, :3] / projected_w
    
    # Coordenadas de pantalla (solo x,y)
    screen_pos = ndc[:, :2]
    
    return screen_pos

# Crear gaussianas con valores predeterminados que aseguran visibilidad
def create_gaussians(pos_x, pos_y, pos_z, scale_x, scale_y, scale_z, rot_x, rot_y, rot_z, rot_w, opacity, color_r, color_g, color_b,skew_x=0,skew_y=0,skew_z=0):
    # Posición
    means3D = torch.tensor([[pos_x, pos_y, pos_z]], dtype=torch.float32, device=device)
    
    # Escala - Asegurarse de que las escalas sean positivas y no demasiado pequeñas
    scale_x = max(scale_x, 0.05)  # Valor mínimo para garantizar visibilidad
    scale_y = max(scale_y, 0.05)
    scale_z = max(scale_z, 0.05)
    scales = torch.tensor([[scale_x, scale_y, scale_z]], dtype=torch.float32, device=device)
    skew= torch.tensor([[skew_x, skew_y, skew_z]], dtype=torch.float32, device=device)
    # Rotación (representada como un cuaternión)
    rot_length = np.sqrt(rot_x**2 + rot_y**2 + rot_z**2 + rot_w**2)
    if rot_length > 0.001:  # Asegurar que no sea casi cero
        rot_normalized = [rot_x/rot_length, rot_y/rot_length, rot_z/rot_length, rot_w/rot_length]
    else:
        rot_normalized = [0, 0, 0, 1]  # Cuaternión identidad si la longitud es casi 0
    
    rotations = torch.tensor([rot_normalized], dtype=torch.float32, device=device)
    
    # Opacidad - Asegurar que sea visible
    opacity = max(opacity, 0.5)  # Mínimo 0.5 para asegurar visibilidad
    opacities = torch.tensor([[opacity]], dtype=torch.float32, device=device)
    
    # Color - Asegurar que no sea negro (sería invisible en fondo negro)
    if color_r == 0 and color_g == 0 and color_b == 0:
        color_r = 1.0  # Predeterminado a rojo si es negro
    
    colors_precomp = torch.tensor([[color_r, color_g, color_b]], dtype=torch.float32, device=device)
    
    return means3D, scales, skew, rotations, opacities, colors_precomp
# Función para renderizar la escena
def render_scene(
    # Cámara
    camera_x=0.0, camera_y=0.0, camera_z=2.0,
    look_x=0.0, look_y=0.0, look_z=0.0,
    up_x=0.0, up_y=1.0, up_z=0.0,
    fov_degrees=45.0, 
    
    # Gaussiana
    pos_x=0.0, pos_y=0.0, pos_z=0.0,
    scale_x=0.2, scale_y=0.2, scale_z=0.2,  # Valores más grandes por defecto
    rot_x=0.0, rot_y=0.0, rot_z=0.0, rot_w=1.0,
    skew_x=0,skew_y=0,skew_z=0,
    opacity=1.0,
    
    # Visualización
    color_r=1.0, color_g=0.0, color_b=0.0,
    bg_r=0.0, bg_g=0.0, bg_b=0.0,
    scale_modifier=1.0,
    image_size=256
):
    try:
        # Print para depuración
        print(f"Renderizando con: Gaussiana en ({pos_x}, {pos_y}, {pos_z}), Escala: ({scale_x}, {scale_y}, {scale_z}), Color: ({color_r}, {color_g}, {color_b})")
        
        # Configurar rasterizador
        bg_color = [bg_r, bg_g, bg_b]
        raster_settings, eye, target, up = create_raster_settings(
            camera_x, camera_y, camera_z,
            look_x, look_y, look_z,
            up_x, up_y, up_z, 
            fov_degrees, bg_color, scale_modifier, image_size
        )
        
        # Crear el rasterizador
        rasterizer = GaussianRasterizer(raster_settings)
        
        # Crear gaussianas
        means3D, scales, skew, rotations, opacities, colors_precomp= create_gaussians(
            pos_x, pos_y, pos_z, 
            scale_x, scale_y, scale_z, 
            rot_x, rot_y, rot_z, rot_w, 
            opacity, 
            color_r, color_g, color_b,
            skew_x, skew_y, skew_z
        )
        
        # Proyectar puntos 3D a 2D
        means2D = project_points(means3D, raster_settings.viewmatrix, raster_settings.projmatrix)
        
        # Imprimir valores para depuración
        print(f"means3D: {means3D}")
        print(f"means2D: {means2D}")
        print(f"scales: {scales}")
        print(f"skew: {skew}")
        print(f"opacities: {opacities}")
        print(f"colors_precomp: {colors_precomp}")
        
        # Renderizar la escena
        rendered_image, depth = rasterizer(
            means3D=means3D,
            means2D=means2D,
            opacities=opacities,
            colors_precomp=colors_precomp,
            scales=scales,
            rotations=rotations,
            skews=skew
        )
        
        # Mostrar imagen
        rendered_image_np = rendered_image.detach().cpu().numpy()
        rendered_image_np = np.transpose(rendered_image_np, (1, 2, 0))  # CHW -> HWC
        rendered_image_np = np.clip(rendered_image_np, 0, 1)
        
        # Comprobar la forma de depth antes de intentar visualizarlo
        if depth is not None and depth.ndim > 1:
            depth_np = depth.detach().cpu().numpy()
            
            plt.figure(figsize=(16, 8))
            
            plt.subplot(1, 2, 1)
            plt.imshow(rendered_image_np)
            plt.axis('off')
            plt.title('Gaussiana 3D Renderizada')
            
            plt.subplot(1, 2, 2)
            plt.imshow(depth_np, cmap='viridis')
            plt.axis('off')
            plt.title('Mapa de Profundidad')
            
            plt.tight_layout()
            plt.show()
        else:
            # Si depth no es una imagen válida, mostrar solo la imagen renderizada
            plt.figure(figsize=(8, 8))
            plt.imshow(rendered_image_np)
            plt.axis('off')
            plt.title('Gaussiana 3D Renderizada')
            plt.show()
            
            if depth is not None:
                print(f"Información de profundidad disponible pero no es una imagen (forma: {depth.shape})")
            else:
                print("No hay información de profundidad disponible")
        
        # Mostrar información de la cámara
        print(f"Cámara: Posición ({camera_x:.2f}, {camera_y:.2f}, {camera_z:.2f})")
        print(f"Mirando a: ({look_x:.2f}, {look_y:.2f}, {look_z:.2f})")
        print(f"Vector up: ({up_x:.2f}, {up_y:.2f}, {up_z:.2f})")
        
        # Comprobar si hay algo visible
        if rendered_image_np.max() <= bg_r and rendered_image_np.max() <= bg_g and rendered_image_np.max() <= bg_b:
            print("ADVERTENCIA: La imagen parece estar completamente en color de fondo. La gaussiana podría no ser visible.")
            print("Sugerencias: Aumenta la escala, modifica la posición, cambia el color o ajusta la opacidad.")
    
    except Exception as e:
        print(f"Error al renderizar: {str(e)}")
        import traceback
        traceback.print_exc()

# Crear widgets para interactuar con la escena con valores predeterminados mejorados
def create_interactive_scene():
    # Controles de cámara
    camera_x = FloatSlider(value=0.0, min=-5.0, max=5.0, step=0.1, description='Cámara X:')
    camera_y = FloatSlider(value=0.0, min=-5.0, max=5.0, step=0.1, description='Cámara Y:')
    camera_z = FloatSlider(value=2.0, min=0.1, max=10.0, step=0.1, description='Cámara Z:')
    
    look_x = FloatSlider(value=0.0, min=-5.0, max=5.0, step=0.1, description='Mirar X:')
    look_y = FloatSlider(value=0.0, min=-5.0, max=5.0, step=0.1, description='Mirar Y:')
    look_z = FloatSlider(value=0.0, min=-5.0, max=5.0, step=0.1, description='Mirar Z:')
    
    # Controles para el vector "up" (normalmente no se cambia mucho)
    up_x = FloatSlider(value=0.0, min=-1.0, max=1.0, step=0.1, description='Up X:')
    up_y = FloatSlider(value=1.0, min=-1.0, max=1.0, step=0.1, description='Up Y:')
    up_z = FloatSlider(value=0.0, min=-1.0, max=1.0, step=0.1, description='Up Z:')
    
    # FOV
    fov = FloatSlider(value=45.0, min=10.0, max=120.0, step=1.0, description='FOV (°):')
    
    # Controles de gaussianas - Valores predeterminados ajustados para mayor visibilidad
    pos_x = FloatSlider(value=0.0, min=-5.0, max=5.0, step=0.05, description='Pos X:')
    pos_y = FloatSlider(value=0.0, min=-5.0, max=5.0, step=0.05, description='Pos Y:')
    pos_z = FloatSlider(value=0.0, min=-5.0, max=5.0, step=0.05, description='Pos Z:')
    
    scale_x = FloatSlider(value=0.2, min=0.05, max=1.0, step=0.01, description='Escala X:')
    scale_y = FloatSlider(value=0.2, min=0.05, max=1.0, step=0.01, description='Escala Y:')
    scale_z = FloatSlider(value=0.2, min=0.05, max=1.0, step=0.01, description='Escala Z:')
    

    skew_x = FloatSlider(value=0.2, min=-1.0, max=1.0, step=0.01, description='Skew X:')
    skew_y = FloatSlider(value=0.2, min=-1.0, max=1.0, step=0.01, description='Skew Y:')
    skew_z = FloatSlider(value=0.2, min=-1.0, max=1.0, step=0.01, description='Skew Z:')

    # Para simplificar, usamos controles más simples para la rotación
    rot_x = FloatSlider(value=0.0, min=-1.0, max=1.0, step=0.1, description='Rot X:')
    rot_y = FloatSlider(value=0.0, min=-1.0, max=1.0, step=0.1, description='Rot Y:')
    rot_z = FloatSlider(value=0.0, min=-1.0, max=1.0, step=0.1, description='Rot Z:')
    rot_w = FloatSlider(value=1.0, min=-1.0, max=1.0, step=0.1, description='Rot W:')
    
    opacity = FloatSlider(value=1.0, min=0.1, max=1.0, step=0.05, description='Opacidad:')
    
    # Colores
    color_r = FloatSlider(value=1.0, min=0.0, max=1.0, step=0.05, description='Color R:')
    color_g = FloatSlider(value=0.0, min=0.0, max=1.0, step=0.05, description='Color G:')
    color_b = FloatSlider(value=0.0, min=0.0, max=1.0, step=0.05, description='Color B:')
    
    bg_r = FloatSlider(value=0.0, min=0.0, max=1.0, step=0.05, description='Fondo R:')
    bg_g = FloatSlider(value=0.0, min=0.0, max=1.0, step=0.05, description='Fondo G:')
    bg_b = FloatSlider(value=0.0, min=0.0, max=1.0, step=0.05, description='Fondo B:')
    
    scale_modifier = FloatSlider(value=2.0, min=0.1, max=10.0, step=0.1, description='Escala global:')
    image_size = IntSlider(value=256, min=128, max=512, step=32, description='Resolución:')
    
    # Crear widget interactivo
    interactive_plot = interactive(
        render_scene,
        camera_x=camera_x, camera_y=camera_y, camera_z=camera_z,
        look_x=look_x, look_y=look_y, look_z=look_z,
        up_x=up_x, up_y=up_y, up_z=up_z,
        fov_degrees=fov,
        pos_x=pos_x, pos_y=pos_y, pos_z=pos_z,
        scale_x=scale_x, scale_y=scale_y, scale_z=scale_z,
        rot_x=rot_x, rot_y=rot_y, rot_z=rot_z, rot_w=rot_w,
        skew_x=skew_x,skew_y=skew_y,skew_z=skew_z,
        opacity=opacity,
        color_r=color_r, color_g=color_g, color_b=color_b,
        bg_r=bg_r, bg_g=bg_g, bg_b=bg_b,
        scale_modifier=scale_modifier,
        image_size=image_size
    )
    
    # Organizar los widgets en pestañas para una mejor organización
    camera_tab = widgets.VBox([
        widgets.HBox([camera_x, camera_y, camera_z]),
        widgets.HBox([look_x, look_y, look_z]),
        widgets.HBox([up_x, up_y, up_z]),
        fov
    ])
    
    gaussian_tab = widgets.VBox([
        widgets.HBox([pos_x, pos_y, pos_z]),
        widgets.HBox([scale_x, scale_y, scale_z]),
        widgets.HBox([skew_x, skew_y, skew_z]),
        widgets.HBox([rot_x, rot_y, rot_z, rot_w]),
        opacity
    ])
    
    visual_tab = widgets.VBox([
        widgets.HBox([color_r, color_g, color_b]),
        widgets.HBox([bg_r, bg_g, bg_b]),
        widgets.HBox([scale_modifier, image_size])
    ])
    
    tab = widgets.Tab(children=[camera_tab, gaussian_tab, visual_tab])
    tab.set_title(0, 'Cámara')
    tab.set_title(1, 'Gaussiana')
    tab.set_title(2, 'Visual')
    
    # Mostrar widgets
    display(tab)
    display(interactive_plot.children[-1])  # Mostrar solo el output
    
    # Conectar los controles al widget interactivo
    return interactive_plot

# Función para ejecutar todo el pipeline
def run_interactive_3d_gaussian():
    print("Inicializando visualizador interactivo de gaussianas 3D...")
    print("Configuración predeterminada: Gaussiana roja con escala 0.2 en el origen.")
    interactive_widget = create_interactive_scene()
    print("¡Listo! Usa los controles para modificar la visualización.")
    print("Si la gaussiana no es visible:")
    print("1. Asegúrate de que el color de la gaussiana sea diferente al color de fondo")
    print("2. Incrementa los valores de escala (X, Y, Z)")
    print("3. Aumenta el valor de 'Escala global'")
    print("4. Ajusta la posición de la cámara")
    return interactive_widget

# Ejecutar
interactive_widget = run_interactive_3d_gaussian()

# Agregar algunas presets predefinidas
def set_orbit_camera(theta, phi, radius):
    """Configura una cámara en órbita alrededor del origen"""
    # Convertir coordenadas esféricas a cartesianas
    x = radius * np.sin(phi) * np.cos(theta)
    y = radius * np.sin(phi) * np.sin(theta)
    z = radius * np.cos(phi)
    
    # Actualizar los widgets
    interactive_widget.kwargs['camera_x'] = x
    interactive_widget.kwargs['camera_y'] = y
    interactive_widget.kwargs['camera_z'] = z
    interactive_widget.kwargs['look_x'] = 0.0
    interactive_widget.kwargs['look_y'] = 0.0
    interactive_widget.kwargs['look_z'] = 0.0
    
    # Volver a renderizar
    interactive_widget.update()

# Configuración preestablecida que funciona para asegurar visibilidad
def set_working_configuration():
    # Configuración de gaussiana grande y colorida
    interactive_widget.kwargs['scale_x'] = 0.5
    interactive_widget.kwargs['scale_y'] = 0.5
    interactive_widget.kwargs['scale_z'] = 0.5
    interactive_widget.kwargs['color_r'] = 1.0
    interactive_widget.kwargs['color_g'] = 0.5
    interactive_widget.kwargs['color_b'] = 0.0
    interactive_widget.kwargs['opacity'] = 1.0
    interactive_widget.kwargs['scale_modifier'] = 5.0
    
    # Cámara a distancia adecuada
    interactive_widget.kwargs['camera_x'] = 0.0
    interactive_widget.kwargs['camera_y'] = 0.0
    interactive_widget.kwargs['camera_z'] = 3.0
    
    # Volver a renderizar
    interactive_widget.update()

# Botones para presets
def create_preset_buttons():
    btn_front = widgets.Button(description="Vista Frontal")
    btn_top = widgets.Button(description="Vista Superior")
    btn_side = widgets.Button(description="Vista Lateral")
    btn_isometric = widgets.Button(description="Vista Isométrica")
    btn_working = widgets.Button(description="Config. Funcional", 
                                button_style='success')
    
    # Asignar funciones a los botones
    btn_front.on_click(lambda b: set_orbit_camera(0, np.pi/2, 2.0))
    btn_top.on_click(lambda b: set_orbit_camera(0, 0, 2.0))
    btn_side.on_click(lambda b: set_orbit_camera(np.pi/2, np.pi/2, 2.0))
    btn_isometric.on_click(lambda b: set_orbit_camera(np.pi/4, np.pi/3, 2.0))
    btn_working.on_click(lambda b: set_working_configuration())
    
    # Mostrar botones con el botón de configuración funcional destacado
    display(widgets.HBox([btn_front, btn_top, btn_side, btn_isometric, btn_working]))

# Mostrar botones de preset
create_preset_buttons()

# Función para crear una animación de rotación
def create_orbit_animation(steps=50):
    from IPython.display import clear_output
    import time
    
    # Asegurarse de tener una configuración que muestre algo visible
    set_working_configuration()
    
    for i in range(steps):
        # Calcular ángulo theta para una rotación completa
        theta = 2 * np.pi * i / steps
        
        # Configurar cámara en órbita
        set_orbit_camera(theta, np.pi/4, 3.0)
        
        # Pequeña pausa para ver la animación
        time.sleep(0.1)

# Botón para iniciar la animación
animation_button = widgets.Button(description="Animar Rotación")
animation_button.on_click(lambda b: create_orbit_animation())
display(animation_button)

Inicializando visualizador interactivo de gaussianas 3D...
Configuración predeterminada: Gaussiana roja con escala 0.2 en el origen.


Output()

¡Listo! Usa los controles para modificar la visualización.
Si la gaussiana no es visible:
1. Asegúrate de que el color de la gaussiana sea diferente al color de fondo
2. Incrementa los valores de escala (X, Y, Z)
3. Aumenta el valor de 'Escala global'
4. Ajusta la posición de la cámara


Button(description='Animar Rotación', style=ButtonStyle())